# Feature Elimination

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='white')

import sklearn
import nltk

import json
import ssl

from collections import Counter
from itertools import chain 
from nltk.tokenize import word_tokenize

In [2]:
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
    
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/wu_ziang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Load the selected data

In [3]:
tmdb_df = pd.read_csv('output/tmdb_5000_selected.csv')
tmdb_df.head()

,id,title,original_language,overview,popularity,release_date,runtime,vote_average,vote_count,cast_num,...,production_company_0_id,production_company_0_name,production_company_1_id,production_company_1_name,production_countries_num,production_country_code,production_country_name,spoken_languages_num,spoken_language_code,spoken_language_name
0,19995,Avatar,en,"In the 22nd century, a paraplegic Marine is di...",150.437577,2009-12-10,162.0,7.2,11800,83,...,289.0,Ingenious Film Partners,306.0,Twentieth Century Fox Film Corporation,2,US,United States of America,2,en,English
1,285,Pirates of the Caribbean: At World's End,en,"Captain Barbossa, long believed to be dead, ha...",139.082615,2007-05-19,169.0,6.9,4500,34,...,2.0,Walt Disney Pictures,130.0,Jerry Bruckheimer Films,1,US,United States of America,1,en,English
2,206647,Spectre,en,A cryptic message from Bond’s past sends him o...,107.376788,2015-10-26,148.0,6.3,4466,83,...,5.0,Columbia Pictures,10761.0,Danjaq,2,GB,United Kingdom,5,fr,Français
3,49026,The Dark Knight Rises,en,Following the death of District Attorney Harve...,112.312950,2012-07-16,165.0,7.6,9106,158,...,923.0,Legendary Pictures,6194.0,Warner Bros.,1,US,United States of America,1,en,English
4,49529,John Carter,en,"John Carter is a war-weary, former military ca...",43.926995,2012-03-07,132.0,6.1,2124,27,...,2.0,Walt Disney Pictures,NaN,NaN,1,US,United States of America,1,en,English


In [4]:
tmdb_df.columns

Index(['id', 'title', 'original_language', 'overview', 'popularity',
       'release_date', 'runtime', 'vote_average', 'vote_count', 'cast_num',
       'cast_0_id', 'cast_0_name', 'cast_0_gender', 'cast_1_id', 'cast_1_name',
       'cast_1_gender', 'cast_2_id', 'cast_2_name', 'cast_2_gender',
       'cast_gender_na', 'cast_gender_f', 'cast_gender_m', 'crew_num',
       'director_id', 'director_name', 'genre_0_id', 'genre_0_name',
       'genre_1_id', 'genre_1_name', 'genre_2_id', 'genre_2_name',
       'keyword_0_id', 'keyword_0_name', 'keyword_1_id', 'keyword_1_name',
       'keyword_2_id', 'keyword_2_name', 'keyword_3_id', 'keyword_3_name',
       'keyword_4_id', 'keyword_4_name', 'production_companies_num',
       'production_company_0_id', 'production_company_0_name',
       'production_company_1_id', 'production_company_1_name',
       'production_countries_num', 'production_country_code',
       'production_country_name', 'spoken_languages_num',
       'spoken_language_code', 'sp

In [5]:
tmdb_df.shape

(4791, 52)

### 1.1 Uniqueness of Cast Names

In [6]:
cast_columns = [['cast_'+str(i)+'_id', 'cast_'+str(i)+'_name', 'cast_'+str(i)+'_gender'] for i in range(3)]
cast_columns = list(chain.from_iterable(cast_columns))

tmdb_df_cast = tmdb_df[['id', 'title']+cast_columns].copy()
tmdb_df_cast

,id,title,cast_0_id,cast_0_name,cast_0_gender,cast_1_id,cast_1_name,cast_1_gender,cast_2_id,cast_2_name,cast_2_gender
0,19995,Avatar,65731.0,Sam Worthington,2.0,8691.0,Zoe Saldana,1.0,10205.0,Sigourney Weaver,1.0
1,285,Pirates of the Caribbean: At World's End,85.0,Johnny Depp,2.0,114.0,Orlando Bloom,2.0,116.0,Keira Knightley,1.0
2,206647,Spectre,8784.0,Daniel Craig,2.0,27319.0,Christoph Waltz,2.0,121529.0,Léa Seydoux,1.0
3,49026,The Dark Knight Rises,3894.0,Christian Bale,2.0,3895.0,Michael Caine,2.0,64.0,Gary Oldman,2.0
4,49529,John Carter,60900.0,Taylor Kitsch,2.0,21044.0,Lynn Collins,1.0,2206.0,Samantha Morton,1.0
...,...,...,...,...,...,...,...,...,...,...,...
4786,9367,El Mariachi,20498.0,Carlos Gallardo,2.0,57470.0,Jaime de Hoyos,0.0,57471.0,Peter Marquardt,2.0
4787,72766,Newlyweds,12833.0,Edward Burns,2.0,58356.0,Kerry Bishé,1.0,17188.0,Marsha Dietlein,1.0
4788,231617,"Signed, Sealed, Delivered",11867.0,Eric Mabius,2.0,34915.0,Kristin Booth,1.0,58393.0,Crystal Lowe,1.0
4789,126186,Shanghai Calling,82093.0,Daniel Henney,2.0,210050.0,Eliza Coupe,1.0,2053.0,Bill Paxton,2.0


In [7]:
tmdb_df_cast_stack = tmdb_df_cast.set_index(['id', 'title']).stack().reset_index().rename({'level_2': 'cast'}, axis=1)
tmdb_df_cast_stack = tmdb_df_cast_stack.rename(columns={'id':'movie_id'})

tmdb_df_cast_stack.head(9)

,movie_id,title,cast,0
0,19995,Avatar,cast_0_id,65731
1,19995,Avatar,cast_0_name,Sam Worthington
2,19995,Avatar,cast_0_gender,2
3,19995,Avatar,cast_1_id,8691
4,19995,Avatar,cast_1_name,Zoe Saldana
5,19995,Avatar,cast_1_gender,1
6,19995,Avatar,cast_2_id,10205
7,19995,Avatar,cast_2_name,Sigourney Weaver
8,19995,Avatar,cast_2_gender,1


In [8]:
def merge_id_name_gender(df):
    df_merge = []

    for key in ['id', 'name', 'gender']:
        df_key = df[df['cast'].str.contains(key)].copy()

        df_key.rename({0: key}, axis=1, inplace=True)
        df_key['cast']=df_key['cast'].str[5].astype(int)

        df_merge.append(df_key)
    
    return pd.merge(pd.merge(df_merge[0], df_merge[1],  on=['movie_id', 'title', 'cast']), 
                    df_merge[2],  on=['movie_id', 'title', 'cast'])

In [9]:
cast_df = merge_id_name_gender(tmdb_df_cast_stack)
cast_df.head(15)

,movie_id,title,cast,id,name,gender
0,19995,Avatar,0,65731,Sam Worthington,2
1,19995,Avatar,1,8691,Zoe Saldana,1
2,19995,Avatar,2,10205,Sigourney Weaver,1
3,285,Pirates of the Caribbean: At World's End,0,85,Johnny Depp,2
4,285,Pirates of the Caribbean: At World's End,1,114,Orlando Bloom,2
5,285,Pirates of the Caribbean: At World's End,2,116,Keira Knightley,1
6,206647,Spectre,0,8784,Daniel Craig,2
7,206647,Spectre,1,27319,Christoph Waltz,2
8,206647,Spectre,2,121529,Léa Seydoux,1
9,49026,The Dark Knight Rises,0,3894,Christian Bale,2


In [10]:
cast_df = cast_df[['id', 'name', 'gender', 'movie_id', 'title', 'cast']]
cast_df = cast_df.rename(columns={'cast':'order'})
cast_df.head(15)

,id,name,gender,movie_id,title,order
0,65731,Sam Worthington,2,19995,Avatar,0
1,8691,Zoe Saldana,1,19995,Avatar,1
2,10205,Sigourney Weaver,1,19995,Avatar,2
3,85,Johnny Depp,2,285,Pirates of the Caribbean: At World's End,0
4,114,Orlando Bloom,2,285,Pirates of the Caribbean: At World's End,1
5,116,Keira Knightley,1,285,Pirates of the Caribbean: At World's End,2
6,8784,Daniel Craig,2,206647,Spectre,0
7,27319,Christoph Waltz,2,206647,Spectre,1
8,121529,Léa Seydoux,1,206647,Spectre,2
9,3894,Christian Bale,2,49026,The Dark Knight Rises,0


In [11]:
cast_df.groupby('name')['id'].nunique().sort_values(ascending=False)

name
Steve James            2
Юлия Снигирь           1
Ginnifer Goodwin       1
Gina Carano            1
Gina Gershon           1
                      ..
Michael Davis          1
Michael Des Barres     1
Michael Douglas        1
Michael Dudikoff       1
"Weird Al" Yankovic    1
Name: id, Length: 5669, dtype: int64

In [12]:
cast_df[cast_df['name']=='Steve James']

,id,name,gender,movie_id,title,order
13369,122449,Steve James,2,14275,Hoop Dreams,2
13680,55271,Steve James,2,25678,American Ninja 2: The Confrontation,1


In [13]:
cast_df.to_csv('output/cast.csv', index=False)

### 1.2 Uniqueness of Director Names

In [14]:
crew_columns = ['director_id', 'director_name']

tmdb_df_crew = tmdb_df[['id', 'title']+crew_columns].copy()
tmdb_df_crew

,id,title,director_id,director_name
0,19995,Avatar,2710.0,James Cameron
1,285,Pirates of the Caribbean: At World's End,1704.0,Gore Verbinski
2,206647,Spectre,39.0,Sam Mendes
3,49026,The Dark Knight Rises,525.0,Christopher Nolan
4,49529,John Carter,7.0,Andrew Stanton
...,...,...,...,...
4786,9367,El Mariachi,2294.0,Robert Rodriguez
4787,72766,Newlyweds,12833.0,Edward Burns
4788,231617,"Signed, Sealed, Delivered",1219158.0,Scott Smith
4789,126186,Shanghai Calling,208138.0,Daniel Hsia


In [15]:
tmdb_df_crew.groupby('director_name')['director_id'].nunique()

director_name
Aaron Hann            1
Aaron Schneider       1
Abel Ferrara          1
Adam Brooks           1
Adam Carolla          1
                     ..
Àlex Pastor           1
Álex de la Iglesia    1
Émile Gaudreault      1
Éric Tessier          1
Étienne Faure         1
Name: director_id, Length: 2340, dtype: int64

In [16]:
director_df = tmdb_df_crew[['director_id', 'director_name', 'id', 'title']]
director_df = director_df.rename(columns={'director_id':'id', 'director_name':'name', 'id':'movie_id'})
director_df.head(15)

,id,name,movie_id,title
0,2710.0,James Cameron,19995,Avatar
1,1704.0,Gore Verbinski,285,Pirates of the Caribbean: At World's End
2,39.0,Sam Mendes,206647,Spectre
3,525.0,Christopher Nolan,49026,The Dark Knight Rises
4,7.0,Andrew Stanton,49529,John Carter
5,7623.0,Sam Raimi,559,Spider-Man 3
6,76595.0,Byron Howard,38757,Tangled
7,12891.0,Joss Whedon,99861,Avengers: Age of Ultron
8,11343.0,David Yates,767,Harry Potter and the Half-Blood Prince
9,15217.0,Zack Snyder,209112,Batman v Superman: Dawn of Justice


In [17]:
director_df.to_csv('output/director.csv', index=False)

In [18]:
tmdb_df.drop(['director_id'], axis=1, inplace=True)

### 1.3 Uniqueness of Genres

In [19]:
genres_columns = [['genre_'+str(i)+'_id', 'genre_'+str(i)+'_name'] for i in range(3)]
genres_columns = list(chain.from_iterable(genres_columns))

tmdb_df_genres = tmdb_df[['id', 'title']+genres_columns].copy()
tmdb_df_genres

,id,title,genre_0_id,genre_0_name,genre_1_id,genre_1_name,genre_2_id,genre_2_name
0,19995,Avatar,28.0,Action,12.0,Adventure,14.0,Fantasy
1,285,Pirates of the Caribbean: At World's End,12.0,Adventure,14.0,Fantasy,28.0,Action
2,206647,Spectre,28.0,Action,12.0,Adventure,80.0,Crime
3,49026,The Dark Knight Rises,28.0,Action,80.0,Crime,18.0,Drama
4,49529,John Carter,28.0,Action,12.0,Adventure,878.0,Science Fiction
...,...,...,...,...,...,...,...,...
4786,9367,El Mariachi,28.0,Action,80.0,Crime,53.0,Thriller
4787,72766,Newlyweds,35.0,Comedy,10749.0,Romance,NaN,NaN
4788,231617,"Signed, Sealed, Delivered",35.0,Comedy,18.0,Drama,10749.0,Romance
4789,126186,Shanghai Calling,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
tmdb_df_genres_stack = tmdb_df_genres.set_index(['id', 'title']).stack().reset_index().rename({'level_2': 'genre'}, axis=1)
tmdb_df_genres_stack = tmdb_df_genres_stack.rename(columns={'id':'movie_id'})

tmdb_df_genres_stack.head(6)

,movie_id,title,genre,0
0,19995,Avatar,genre_0_id,28
1,19995,Avatar,genre_0_name,Action
2,19995,Avatar,genre_1_id,12
3,19995,Avatar,genre_1_name,Adventure
4,19995,Avatar,genre_2_id,14
5,19995,Avatar,genre_2_name,Fantasy


In [21]:
def merge_id_name(df, column):
    df_merge = []

    for key in ['id', 'name']:
        df_key = df[df[column].str.contains(key)].copy()

        df_key.rename({0: key}, axis=1, inplace=True)
        df_key[column]=df_key[column].str[len(column)+1].astype(int)

        df_merge.append(df_key)
    
    return pd.merge(df_merge[0], df_merge[1],  on=['movie_id', 'title', column])

In [22]:
genre_df = merge_id_name(tmdb_df_genres_stack, 'genre')
genre_df.head(15)

,movie_id,title,genre,id,name
0,19995,Avatar,0,28,Action
1,19995,Avatar,1,12,Adventure
2,19995,Avatar,2,14,Fantasy
3,285,Pirates of the Caribbean: At World's End,0,12,Adventure
4,285,Pirates of the Caribbean: At World's End,1,14,Fantasy
5,285,Pirates of the Caribbean: At World's End,2,28,Action
6,206647,Spectre,0,28,Action
7,206647,Spectre,1,12,Adventure
8,206647,Spectre,2,80,Crime
9,49026,The Dark Knight Rises,0,28,Action


In [23]:
genre_df = genre_df[['id', 'name', 'movie_id', 'title', 'genre']]
genre_df.head(15)

,id,name,movie_id,title,genre
0,28,Action,19995,Avatar,0
1,12,Adventure,19995,Avatar,1
2,14,Fantasy,19995,Avatar,2
3,12,Adventure,285,Pirates of the Caribbean: At World's End,0
4,14,Fantasy,285,Pirates of the Caribbean: At World's End,1
5,28,Action,285,Pirates of the Caribbean: At World's End,2
6,28,Action,206647,Spectre,0
7,12,Adventure,206647,Spectre,1
8,80,Crime,206647,Spectre,2
9,28,Action,49026,The Dark Knight Rises,0


In [24]:
genre_df.groupby('name')['id'].nunique().sort_values(ascending=False)

name
Western            1
War                1
Adventure          1
Animation          1
Comedy             1
Crime              1
Documentary        1
Drama              1
Family             1
Fantasy            1
Foreign            1
History            1
Horror             1
Music              1
Mystery            1
Romance            1
Science Fiction    1
TV Movie           1
Thriller           1
Action             1
Name: id, dtype: int64

In [25]:
genre_df.to_csv('output/genre.csv', index=False)

In [26]:
tmdb_df.drop(['genre_'+str(i)+'_id' for i in range(3)], axis=1, inplace=True)

### 1.4 Uniqueness of Keywords

In [27]:
keywords_columns = [['keyword_'+str(i)+'_id', 'keyword_'+str(i)+'_name'] for i in range(5)]
keywords_columns = list(chain.from_iterable(keywords_columns))

tmdb_df_keywords = tmdb_df[['id', 'title']+keywords_columns].copy()
tmdb_df_keywords

,id,title,keyword_0_id,keyword_0_name,keyword_1_id,keyword_1_name,keyword_2_id,keyword_2_name,keyword_3_id,keyword_3_name,keyword_4_id,keyword_4_name
0,19995,Avatar,1463.0,culture clash,2964.0,future,3386.0,space war,3388.0,space colony,3679.0,society
1,285,Pirates of the Caribbean: At World's End,270.0,ocean,726.0,drug abuse,911.0,exotic island,1319.0,east india trading company,2038.0,love of one's life
2,206647,Spectre,470.0,spy,818.0,based on novel,4289.0,secret agent,9663.0,sequel,14555.0,mi6
3,49026,The Dark Knight Rises,849.0,dc comics,853.0,crime fighter,949.0,terrorist,1308.0,secret identity,1437.0,burglar
4,49529,John Carter,818.0,based on novel,839.0,mars,1456.0,medallion,3801.0,space travel,7376.0,princess
...,...,...,...,...,...,...,...,...,...,...,...,...
4786,9367,El Mariachi,5616.0,united states–mexico barrier,33649.0,legs,162740.0,arms,187891.0,paper knife,206558.0,guitar case
4787,72766,Newlyweds,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4788,231617,"Signed, Sealed, Delivered",248.0,date,699.0,love at first sight,2398.0,narration,5340.0,investigation,34051.0,team
4789,126186,Shanghai Calling,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
tmdb_df_keywords_stack = tmdb_df_keywords.set_index(['id', 'title']).stack().reset_index().rename({'level_2': 'keyword'}, axis=1)
tmdb_df_keywords_stack = tmdb_df_keywords_stack.rename(columns={'id':'movie_id'})

tmdb_df_keywords_stack.head(9)

,movie_id,title,keyword,0
0,19995,Avatar,keyword_0_id,1463
1,19995,Avatar,keyword_0_name,culture clash
2,19995,Avatar,keyword_1_id,2964
3,19995,Avatar,keyword_1_name,future
4,19995,Avatar,keyword_2_id,3386
5,19995,Avatar,keyword_2_name,space war
6,19995,Avatar,keyword_3_id,3388
7,19995,Avatar,keyword_3_name,space colony
8,19995,Avatar,keyword_4_id,3679


In [29]:
keyword_df = merge_id_name(tmdb_df_keywords_stack, 'keyword')
keyword_df.head(15)

,movie_id,title,keyword,id,name
0,19995,Avatar,0,1463,culture clash
1,19995,Avatar,1,2964,future
2,19995,Avatar,2,3386,space war
3,19995,Avatar,3,3388,space colony
4,19995,Avatar,4,3679,society
5,285,Pirates of the Caribbean: At World's End,0,270,ocean
6,285,Pirates of the Caribbean: At World's End,1,726,drug abuse
7,285,Pirates of the Caribbean: At World's End,2,911,exotic island
8,285,Pirates of the Caribbean: At World's End,3,1319,east india trading company
9,285,Pirates of the Caribbean: At World's End,4,2038,love of one's life


In [30]:
keyword_df = keyword_df[['id', 'name', 'movie_id', 'title', 'keyword']]
keyword_df.head(15)

,id,name,movie_id,title,keyword
0,1463,culture clash,19995,Avatar,0
1,2964,future,19995,Avatar,1
2,3386,space war,19995,Avatar,2
3,3388,space colony,19995,Avatar,3
4,3679,society,19995,Avatar,4
5,270,ocean,285,Pirates of the Caribbean: At World's End,0
6,726,drug abuse,285,Pirates of the Caribbean: At World's End,1
7,911,exotic island,285,Pirates of the Caribbean: At World's End,2
8,1319,east india trading company,285,Pirates of the Caribbean: At World's End,3
9,2038,love of one's life,285,Pirates of the Caribbean: At World's End,4


In [31]:
keyword_df.groupby('name')['id'].nunique().sort_values(ascending=False)

name
超级妈妈                       1
family clan                1
film history               1
film in a film             1
film industry              1
                          ..
planet                     1
planetarium                1
planetary configuration    1
planned murder             1
15th century               1
Name: id, Length: 5133, dtype: int64

In [32]:
keyword_df.to_csv('output/keyword.csv', index=False)

In [33]:
tmdb_df.drop(['keyword_'+str(i)+'_id' for i in range(5)], axis=1, inplace=True)

### 1.5 Uniqueness of Production Companies

In [34]:
pro_com_columns = [['production_company_'+str(i)+'_id', 'production_company_'+str(i)+'_name'] for i in range(2)]
pro_com_columns = list(chain.from_iterable(pro_com_columns))

tmdb_df_pro_com = tmdb_df[['id', 'title']+pro_com_columns].copy()
tmdb_df_pro_com

,id,title,production_company_0_id,production_company_0_name,production_company_1_id,production_company_1_name
0,19995,Avatar,289.0,Ingenious Film Partners,306.0,Twentieth Century Fox Film Corporation
1,285,Pirates of the Caribbean: At World's End,2.0,Walt Disney Pictures,130.0,Jerry Bruckheimer Films
2,206647,Spectre,5.0,Columbia Pictures,10761.0,Danjaq
3,49026,The Dark Knight Rises,923.0,Legendary Pictures,6194.0,Warner Bros.
4,49529,John Carter,2.0,Walt Disney Pictures,NaN,NaN
...,...,...,...,...,...,...
4786,9367,El Mariachi,5.0,Columbia Pictures,NaN,NaN
4787,72766,Newlyweds,NaN,NaN,NaN,NaN
4788,231617,"Signed, Sealed, Delivered",3958.0,Front Street Pictures,6438.0,Muse Entertainment Enterprises
4789,126186,Shanghai Calling,NaN,NaN,NaN,NaN


In [35]:
tmdb_df_pro_com_stack = tmdb_df_pro_com.set_index(['id', 'title']).stack().reset_index().rename({'level_2': 'production_company'}, axis=1)
tmdb_df_pro_com_stack = tmdb_df_pro_com_stack.rename(columns={'id':'movie_id'})

tmdb_df_pro_com_stack.head(8)

,movie_id,title,production_company,0
0,19995,Avatar,production_company_0_id,289
1,19995,Avatar,production_company_0_name,Ingenious Film Partners
2,19995,Avatar,production_company_1_id,306
3,19995,Avatar,production_company_1_name,Twentieth Century Fox Film Corporation
4,285,Pirates of the Caribbean: At World's End,production_company_0_id,2
5,285,Pirates of the Caribbean: At World's End,production_company_0_name,Walt Disney Pictures
6,285,Pirates of the Caribbean: At World's End,production_company_1_id,130
7,285,Pirates of the Caribbean: At World's End,production_company_1_name,Jerry Bruckheimer Films


In [36]:
pro_com_df = merge_id_name(tmdb_df_pro_com_stack, 'production_company')
pro_com_df.head(15)

,movie_id,title,production_company,id,name
0,19995,Avatar,0,289,Ingenious Film Partners
1,19995,Avatar,1,306,Twentieth Century Fox Film Corporation
2,285,Pirates of the Caribbean: At World's End,0,2,Walt Disney Pictures
3,285,Pirates of the Caribbean: At World's End,1,130,Jerry Bruckheimer Films
4,206647,Spectre,0,5,Columbia Pictures
5,206647,Spectre,1,10761,Danjaq
6,49026,The Dark Knight Rises,0,923,Legendary Pictures
7,49026,The Dark Knight Rises,1,6194,Warner Bros.
8,49529,John Carter,0,2,Walt Disney Pictures
9,559,Spider-Man 3,0,5,Columbia Pictures


In [37]:
pro_com_df = pro_com_df[['id', 'name', 'movie_id', 'title', 'production_company']]
pro_com_df.head(15)

,id,name,movie_id,title,production_company
0,289,Ingenious Film Partners,19995,Avatar,0
1,306,Twentieth Century Fox Film Corporation,19995,Avatar,1
2,2,Walt Disney Pictures,285,Pirates of the Caribbean: At World's End,0
3,130,Jerry Bruckheimer Films,285,Pirates of the Caribbean: At World's End,1
4,5,Columbia Pictures,206647,Spectre,0
5,10761,Danjaq,206647,Spectre,1
6,923,Legendary Pictures,49026,The Dark Knight Rises,0
7,6194,Warner Bros.,49026,The Dark Knight Rises,1
8,2,Walt Disney Pictures,49529,John Carter,0
9,5,Columbia Pictures,559,Spider-Man 3,0


In [38]:
pro_com_df.groupby('name')['id'].nunique().sort_values(ascending=False).head(10)

name
Esparza / Katz Productions                2
HSX Films                                 2
British Broadcasting Corporation (BBC)    2
Ignite Entertainment                      2
3 Arts Entertainment                      2
Anonymous Content                         2
Automatik Entertainment                   2
France 2 Cinéma                           2
Focus Features                            1
Focus Films                               1
Name: id, dtype: int64

In [39]:
pro_com_df[pro_com_df['name']=='Esparza / Katz Productions']

,id,name,movie_id,title,production_company
3546,26560,Esparza / Katz Productions,10655,Gettysburg,1
3907,9313,Esparza / Katz Productions,16052,Selena,1


In [40]:
pro_com_df[pro_com_df['name']=='HSX Films']

,id,name,movie_id,title,production_company
6404,15461,HSX Films,41730,Desert Blue,1
6979,4714,HSX Films,78373,"Dancer, Texas Pop. 81",0
7077,4714,HSX Films,24746,Six-String Samurai,0


In [41]:
pro_com_df[pro_com_df['name']=='British Broadcasting Corporation (BBC)']

,id,name,movie_id,title,production_company
5846,3324,British Broadcasting Corporation (BBC),153397,Restless,0
6015,15278,British Broadcasting Corporation (BBC),24985,The Heart of Me,0
6519,3324,British Broadcasting Corporation (BBC),47452,Love and Death on Long Island,1
6614,3324,British Broadcasting Corporation (BBC),125123,Among Giants,1


In [42]:
pro_com_df.to_csv('output/production_company.csv', index=False)

### Uniqueness of Production Countries

In [43]:
pro_coun_columns = ['production_country_code', 'production_country_name']

tmdb_df_pro_coun = tmdb_df[['id', 'title']+pro_coun_columns].copy()
tmdb_df_pro_coun

,id,title,production_country_code,production_country_name
0,19995,Avatar,US,United States of America
1,285,Pirates of the Caribbean: At World's End,US,United States of America
2,206647,Spectre,GB,United Kingdom
3,49026,The Dark Knight Rises,US,United States of America
4,49529,John Carter,US,United States of America
...,...,...,...,...
4786,9367,El Mariachi,MX,Mexico
4787,72766,Newlyweds,NaN,NaN
4788,231617,"Signed, Sealed, Delivered",US,United States of America
4789,126186,Shanghai Calling,US,United States of America


In [44]:
tmdb_df_pro_coun.groupby('production_country_name')['production_country_code'].nunique().sort_values()

production_country_name
Afghanistan                 1
Jamaica                     1
Japan                       1
Kenya                       1
Kyrgyz Republic             1
                           ..
Fiji                        1
Ecuador                     1
Dominican Republic          1
Greece                      1
United States of America    1
Name: production_country_code, Length: 70, dtype: int64

In [45]:
pro_coun_df = tmdb_df_pro_coun[['production_country_code', 'production_country_name', 'id', 'title']]
pro_coun_df = pro_coun_df.rename(columns={'production_country_code':'code', 'production_country_name':'name', 'id':'movie_id'})
pro_coun_df.head(15)

,code,name,movie_id,title
0,US,United States of America,19995,Avatar
1,US,United States of America,285,Pirates of the Caribbean: At World's End
2,GB,United Kingdom,206647,Spectre
3,US,United States of America,49026,The Dark Knight Rises
4,US,United States of America,49529,John Carter
5,US,United States of America,559,Spider-Man 3
6,US,United States of America,38757,Tangled
7,US,United States of America,99861,Avengers: Age of Ultron
8,GB,United Kingdom,767,Harry Potter and the Half-Blood Prince
9,US,United States of America,209112,Batman v Superman: Dawn of Justice


In [46]:
pro_coun_df.to_csv('output/production_country.csv', index=False)

In [47]:
tmdb_df.drop(['production_country_name'], axis=1, inplace=True)

### Uniqueness of Spoken Language

In [48]:
spoken_lang_columns = ['spoken_language_code', 'spoken_language_name']

tmdb_df_spoken_lang = tmdb_df[['id', 'title']+spoken_lang_columns].copy()
tmdb_df_spoken_lang

,id,title,spoken_language_code,spoken_language_name
0,19995,Avatar,en,English
1,285,Pirates of the Caribbean: At World's End,en,English
2,206647,Spectre,fr,Français
3,49026,The Dark Knight Rises,en,English
4,49529,John Carter,en,English
...,...,...,...,...
4786,9367,El Mariachi,es,Español
4787,72766,Newlyweds,NaN,NaN
4788,231617,"Signed, Sealed, Delivered",en,English
4789,126186,Shanghai Calling,en,English


In [49]:
tmdb_df_spoken_lang.groupby('spoken_language_name')['spoken_language_code'].nunique().sort_values()

spoken_language_name
??????              1
Srpski              1
Tiếng Việt          1
shqip               1
svenska             1
Íslenska            1
Český               1
ελληνικά            1
Український         1
български език      1
עִבְרִית            1
اردو                1
العربية             1
فارسی               1
हिन्दी              1
বাংলা               1
தமிழ்               1
ภาษาไทย             1
广州话 / 廣州話           1
日本語                 1
Română              1
Pусский             1
Português           1
Polski              1
Afrikaans           1
Bahasa indonesia    1
Bosanski            1
Dansk               1
Deutsch             1
Eesti               1
English             1
Español             1
Esperanto           1
普通话                 1
Français            1
Galego              1
Hrvatski            1
Italiano            1
Kiswahili           1
Latin               1
Magyar              1
Nederlands          1
No Language         1
Norsk               1
Gaeilge    

In [50]:
spoken_lang_df = tmdb_df_spoken_lang[['spoken_language_code', 'spoken_language_name', 'id', 'title']]
spoken_lang_df = spoken_lang_df.rename(columns={'spoken_language_code':'code', 'spoken_language_name':'name', 'id':'movie_id'})
spoken_lang_df.head(15)

,code,name,movie_id,title
0,en,English,19995,Avatar
1,en,English,285,Pirates of the Caribbean: At World's End
2,fr,Français,206647,Spectre
3,en,English,49026,The Dark Knight Rises
4,en,English,49529,John Carter
5,en,English,559,Spider-Man 3
6,en,English,38757,Tangled
7,en,English,99861,Avengers: Age of Ultron
8,en,English,767,Harry Potter and the Half-Blood Prince
9,en,English,209112,Batman v Superman: Dawn of Justice


In [51]:
spoken_lang_df.to_csv('output/spoken_language.csv', index=False)

In [52]:
tmdb_df.drop(['spoken_language_name'], axis=1, inplace=True)

### Necessity of Gender

In [53]:
cast_df['gender'].value_counts()

2.0    8510
1.0    4596
0.0    1122
Name: gender, dtype: int64

In [54]:
np.sum(tmdb_df['cast_gender_na']>0.5*(tmdb_df['cast_gender_f']+tmdb_df['cast_gender_m']))

1508

In [55]:
tmdb_df.drop(['cast_gender_na', 'cast_gender_f', 'cast_gender_m'], axis=1, inplace=True)

### Necessity of Cast ID

In [56]:
tmdb_df[tmdb_df['id']==14275].iloc[0]

id                                                                       14275
title                                                              Hoop Dreams
original_language                                                           en
overview                     This documentary follows two inner-city Chicag...
popularity                                                             9.18843
release_date                                                        1994-09-12
runtime                                                                    171
vote_average                                                               7.7
vote_count                                                                  87
cast_num                                                                     3
cast_0_id                                                               134119
cast_0_name                                                      William Gates
cast_0_gender                                       

In [57]:
tmdb_df[tmdb_df['id']==25678].iloc[0]

id                                                                       25678
title                                      American Ninja 2: The Confrontation
original_language                                                           en
overview                     On a remote Caribbean island, Army Ranger Joe ...
popularity                                                             5.06111
release_date                                                        1987-05-01
runtime                                                                     90
vote_average                                                               4.9
vote_count                                                                  49
cast_num                                                                    15
cast_0_id                                                                21259
cast_0_name                                                   Michael Dudikoff
cast_0_gender                                       

In [58]:
tmdb_df = tmdb_df.drop([4589])
tmdb_df.drop(['cast_0_id','cast_1_id','cast_2_id'], axis=1, inplace=True)

In [59]:
tmdb_df.shape

(4790, 35)

### Necessity of Other Features

In [60]:
tmdb_df.drop(['production_companies_num', 'production_company_0_id', 'production_company_1_id',
              'production_countries_num','spoken_languages_num', 'spoken_language_code'], axis=1, inplace=True)

In [64]:
tmdb_df.shape

(4790, 29)

### Final Pitch

In [65]:
tmdb_df.columns

Index(['id', 'title', 'original_language', 'overview', 'popularity',
       'release_date', 'runtime', 'vote_average', 'vote_count', 'cast_num',
       'cast_0_name', 'cast_0_gender', 'cast_1_name', 'cast_1_gender',
       'cast_2_name', 'cast_2_gender', 'crew_num', 'director_name',
       'genre_0_name', 'genre_1_name', 'genre_2_name', 'keyword_0_name',
       'keyword_1_name', 'keyword_2_name', 'keyword_3_name', 'keyword_4_name',
       'production_company_0_name', 'production_company_1_name',
       'production_country_code'],
      dtype='object')

In [66]:
tmdb_df = tmdb_df.rename(columns={'original_language':'language',
                                  'cast_0_name':'cast_0', 'cast_1_name':'cast_1', 'cast_2_name':'cast_2',
                                  'director_name':'director',
                                  'genre_0_name':'genre_0', 'genre_1_name':'genre_1', 'genre_2_name':'genre_2',
                                  'keyword_0_name':'keyword_0','keyword_1_name':'keyword_1',
                                  'keyword_2_name':'keyword_2','keyword_3_name':'keyword_3',
                                  'keyword_4_name':'keyword_4',
                                  'production_company_0_name':'company_0', 'production_company_1_name':'company_1',
                                  'production_country_code':'country'})

In [69]:
tmdb_df.columns

Index(['id', 'title', 'language', 'overview', 'popularity', 'release_date',
       'runtime', 'vote_average', 'vote_count', 'cast_num', 'cast_0',
       'cast_0_gender', 'cast_1', 'cast_1_gender', 'cast_2', 'cast_2_gender',
       'crew_num', 'director', 'genre_0', 'genre_1', 'genre_2', 'keyword_0',
       'keyword_1', 'keyword_2', 'keyword_3', 'keyword_4', 'company_0',
       'company_1', 'country'],
      dtype='object')

In [72]:
tmdb_df.to_csv('data/tmdb_5000_features.csv', index=False)